In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_athens = pd.read_csv('data_athens.csv')

In [3]:
data_athens = data_athens.drop(['Unnamed: 0'], axis=1)

In [4]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_athens))

# Remove rows where any of the columns have a z-score greater than 3
data_athens_cleaned = data_athens[(z_scores < 3).all(axis=1)]

In [5]:
data_athens_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.326277,-0.068423,0.241631,0.102946,0.126150,0.275210,-0.361190,-0.145817,0.487646,0.275809,-0.069303,-0.231659,NaN,NaN,0.083801,0.024158
person_capacity,0.326277,1.000000,-0.049664,0.059044,0.005656,0.072428,0.640218,-0.017229,-0.067756,0.064651,0.023184,-0.048264,-0.038090,NaN,NaN,0.000276,-0.007093
multi,-0.068423,-0.049664,1.000000,-0.457469,-0.029984,-0.022027,-0.020757,0.015801,0.013700,-0.023534,-0.005681,0.035734,0.023260,NaN,NaN,0.056467,-0.017229
biz,0.241631,0.059044,-0.457469,1.000000,-0.034335,-0.120096,-0.043714,-0.151219,-0.111385,0.180025,0.112499,-0.072734,-0.097214,NaN,NaN,0.010218,0.017891
cleanliness_rating,0.102946,0.005656,-0.029984,-0.034335,1.000000,0.563329,-0.000260,0.002637,-0.014886,0.023519,0.032525,-0.018091,-0.033912,NaN,NaN,0.283661,-0.006740
guest_satisfaction_overall,0.126150,0.072428,-0.022027,-0.120096,0.563329,1.000000,0.045200,0.016382,0.021119,0.018967,0.027876,0.002499,-0.055175,NaN,NaN,0.285463,-0.016780
bedrooms,0.275210,0.640218,-0.020757,-0.043714,-0.000260,0.045200,1.000000,0.058362,0.014554,-0.031908,-0.034769,-0.017606,0.017133,NaN,NaN,-0.008273,-0.019818
dist,-0.361190,-0.017229,0.015801,-0.151219,0.002637,0.016382,0.058362,1.000000,0.035071,-0.681837,-0.448835,-0.035838,0.584885,NaN,NaN,-0.052158,-0.013303
metro_dist,-0.145817,-0.067756,0.013700,-0.111385,-0.014886,0.021119,0.014554,0.035071,1.000000,-0.196574,-0.146130,0.311203,0.050341,NaN,NaN,-0.012667,-0.008478
attr_index_norm,0.487646,0.064651,-0.023534,0.180025,0.023519,0.018967,-0.031908,-0.681837,-0.196574,1.000000,0.523678,-0.192263,-0.507258,NaN,NaN,0.077204,0.028786


In [6]:
data_athens_cleaned_reg = data_athens_cleaned.drop(['cleanliness_rating', 'host_is_superhost_True', 'lat', 'lng', 'room_shared_True', 'room_private_True'], axis=1)

In [7]:
x = data_athens_cleaned_reg.drop('realSum', axis=1)
y = data_athens_cleaned_reg.realSum

In [8]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,weekend_yes
count,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000,3328.000000
mean,3.798377,0.268630,0.366887,96.048678,1.288762,1.765123,0.471890,5.463662,9.161199,0.506310
std,1.249672,0.443314,0.482028,4.898308,0.663239,0.867753,0.259522,3.486765,8.937386,0.500035
min,2.000000,0.000000,0.000000,70.000000,0.000000,0.070000,0.020000,1.406422,0.834278,0.000000
25%,3.000000,0.000000,0.000000,94.000000,1.000000,1.150000,0.290000,3.176671,2.586991,0.000000
50%,4.000000,0.000000,0.000000,98.000000,1.000000,1.670000,0.420000,4.491057,6.283867,1.000000
75%,4.000000,1.000000,1.000000,100.000000,2.000000,2.240000,0.590000,6.381240,12.710790,1.000000
max,6.000000,1.000000,1.000000,100.000000,3.000000,4.580000,1.330000,19.675688,42.255760,1.000000


In [9]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     205.6
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:11:09   Log-Likelihood:                -18229.
No. Observations:                3328   AIC:                         3.648e+04
Df Residuals:                    3317   BIC:                         3.655e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [10]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [11]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     205.6
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:11:09   Log-Likelihood:                -18229.
No. Observations:                3328   AIC:                         3.648e+04
Df Residuals:                    3317   BIC:                         3.655e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        149.9002      1.005    149.118      0.0

In [12]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-150.87440168680382

In [13]:
model.score(x_train, y_train)

0.3825991554291487

In [14]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [15]:
mse(y_test,y_pred)

2716.8359066505095

In [16]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

52.12327605447023

In [17]:
##prediction on the train set
mse(y_train,y_pred_train)

3351.902232338582

In [18]:
mae(y_test,y_pred)

35.831849262696764

In [19]:
R2=r2_score(y_test,y_pred)
R2

0.406120026403917

In [20]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.40071620498812

In [21]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.38257778510728035, Test -> 0.40603227447702783


In [22]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.3651384433096093, Test -> 0.3826364151206121


In [23]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 3328 rows and 10 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.27
The coefficient of determination for the train data is R2=1.00


In [24]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [1.50596519e-01 2.65752544e-04 2.16528715e-01 3.50039505e-01
 2.36843195e-01]
Score stats: 0.19 accuracy with a standard deviation of 0.11


In [25]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.192655286730135, 'Linear Regression': 0.3753969923267704, 'KNN': 0.27235996542026775}


In [26]:
data_athens_cleaned.to_excel('data_athens.xlsx')